In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Import and Preparation

In [ ]:
df_train = pd.read_csv('../input/train.csv', index_col='PassengerId')
df_test = pd.read_csv('../input/test.csv', index_col='PassengerId')
df_gender_sub = pd.read_csv("../input/gender_submission.csv", index_col='PassengerId')

In [ ]:
# Storing the target separately
Survived = df_train.loc[:,'Survived']
df_train = df_train.drop(['Survived'], axis=1).copy()

# Saving index for train test split 
train_index = df_train.index
test_index = df_test.index

# Concate the two datasets
df_all = pd.concat([df_train, df_test])

# dont needed anymore
##del df_train
##del df_test

# Create new feature Family true/false
df_all.loc[:,'Family'] = ((df_all['SibSp'] > 0) | (df_all['Parch'] > 0)).replace(True, 1, inplace=False)
df_all.loc[:,'Family'] = df_all.loc[:,'Family'].astype(int)

# Missing Values

In [ ]:
# Function for nullanalysis
def nullAnalysis(df):
    tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})

    tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
    tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100)
                         .T.rename(index={0:'null values (%)'}))
    return tab_info

In [ ]:
# Show the null values
nullAnalysis(df_all)

## Age

In [ ]:
# First 10 datarows where age is null
df_all[df_all.loc[:,'Age'].isnull()].head(10)

In [ ]:
# Average age overall
print("Average age of a passengers: ", round(df_all.loc[:,'Age'].agg('mean'),0))

In [ ]:
# Average age per class
df_all.groupby('Pclass')['Age'].agg('mean')

We will use the average age of the corresponding class to fill the missing passenger ages. This would be the best fit for that problem.

In [ ]:
# Setting the average age of each class for the missing values inside the corresponding class
df_all.loc[(df_all['Age'].isnull()) & (df_all['Pclass'] == 1), ['Age']] = round(df_all.groupby('Pclass')['Age'].agg('mean')[1],0)
df_all.loc[(df_all['Age'].isnull()) & (df_all['Pclass'] == 2), ['Age']] = round(df_all.groupby('Pclass')['Age'].agg('mean')[2],0)
df_all.loc[(df_all['Age'].isnull()) & (df_all['Pclass'] == 3), ['Age']] = round(df_all.groupby('Pclass')['Age'].agg('mean')[3],0)

## Fare


In [ ]:
df_all[df_all['Fare'].isnull()]

One passenger has not payed his Ticket, or is even not recorded. To fill the gab I will use the mean ticket price for his third class ticket: 13.30 

In [ ]:
df_all.groupby('Pclass', as_index=False)['Fare'].agg('mean')

In [ ]:
# Setting Fare to mean fare of pclass
df_all.loc[1044,['Fare']] = 13.30

## Cabin

In [ ]:
# Show all Cabins with NaN data
df_all.loc[(df_all['Cabin'].isnull())].count()

The *Cabin* situation looks pretty bad. From the 1309 data points are only roundabout 300 filled with *Cabin* information, the other 1000 data points are empty. I assumed with the cabin information one could rather conclude on a passengers survival or not. I don't want to use the fillna() - method with this sparely filled feature, that would not make any sense and will probably affect my results badly.  Lets look into the *Cabin* distribution compared the the *Pclass*

In [ ]:
# Group by Pclasses
df_all.groupby('Pclass').agg('count')[['Name','Cabin']]

The round about 300 filled examples for the *Cabin* information are fairly one-sided distributed to the first class (*Pclass*). The *Name* shows the total distribution for the cabin class, so 256 passengers of 323 first class passengers have a *Cabin* information available. Only 39 *Cabin* values are filled in the second and third class (*Pclass*), they are divided up with 23 in second and 16 in third class. In comparision to the amount of passengers makes it not better, there are 709 passengers in the third class and 277 in second class. First class has 323 passengers in it's area. The completeness of the data in relation to the number of passengers per class is as follows:

In [ ]:
(df_all.groupby('Pclass').agg('count')['Cabin'] / df_all.groupby('Pclass').agg('count')['Name'])*100

It clearly shows that the second and third class are sparsely filled with in comparision to their passengers count. The first class in comparision looks with nearly 80% pretty good. One could consider applying the **fillNa** only to the first class and using a different method for the second and third class.

While analysing the *Cabin* feature I realize there are sometimes multiple values inside the *Cabin* feature. The following table shows you the multiple *Cabin* values:

In [ ]:
df_all[df_all['Cabin'].str.contains(' ', regex=False) == True].sort_values(by='Cabin')

There is a way to split the string values easily but the information gain from the 41 examples in comparision to the round about 1000 missing values is pretty low:

In [ ]:
# Creating a data frame for the Cabin values to split multiple values into seperate columns
df_cabin_expand = df_all.loc[:,'Cabin'].str.split(' ', expand=True)

# Group all doubled values by the first value
df_cabin_expand[df_cabin_expand.loc[:,1].isnull() == False].groupby([0]).count()

The *Cabin* feature will be dropped at least. It there is no way of restoring the informaiton from this column, the duplicate values are not enough to restore all missing values and the other features to not supply enough information to restore this based of them.

## Salutation
With analyzing the *Cabin* feature I realized there exists different salutations in the passengers *Name* feature. The string split funciton will be usefull here, mentioned in the *Cabin* feature analysis. The values differ among others as follows:


In [ ]:
# Split Name feature strings into several columns
df_name_salutation = df_all.loc[:,'Name'].str.split(' ', expand=True)
df_name_salutation.groupby(1).count()

Here is still something to clean but you can clearly see, that the most values in the split column "1" refers to the passengers salutation. Moreover I recognized that the salutations strings always ends with a dot.:

- Capt. - Captain
- Col. - Colonel
- Don. - Don
- Dr. - Doctor
- Major.
- Master. - "a way of addressing politely a boy ... too young to be called 'Mister'." - *Leslie Dunkling*  
- Miss. 
- Mlle. - Mademoiselle
- Mme. - Madame
- Mr.
- Mrs.
- Ms.
- Countess.
- ...

Next I will retriev the single salutations from column 1 and 2 based on the string value "." (dot) and concatenate the two columns to one salutation column and append these to the main dataframe. 

In [ ]:
# Extract Salutation from every column based on the '.'
df_newsal_1 = df_name_salutation[df_name_salutation[1].str.contains('.', regex=False)][1]
df_newsal_2 = df_name_salutation[df_name_salutation[2].str.contains('.', regex=False)][2]
df_newsal_3 = df_name_salutation[(df_name_salutation[3].isnull() == False) & (df_name_salutation[3].str.contains('.', regex=False))][3]

# Rename column for append 
df_newsal_2 = df_newsal_2.rename(1)
df_newsal_3 =  df_newsal_3.rename(1)

# Append both salutations results to one column and rename them
df_newsal = df_newsal_1.append([df_newsal_2, df_newsal_3])
df_newsal = df_newsal.rename('Salutation')

# Concatenate them to the main dataframe
df_all = pd.concat([df_all,df_newsal],axis=1)

The following list shows all the salutation distribution. Nothing is leftover right now:

In [ ]:
df_all.groupby('Salutation').count()

# Feature Distribution

## Gender Distribution

In [ ]:
# Gender distribution
df_all.groupby(['Parch']).agg('count')

The feature **Cabin** seems to have some missing values.

## Gender Distribution by Ticket Class
Following will show the gender distribution by ticket class.

In [ ]:
# Visualizing pie chart

fig, ax = plt.subplots(figsize=(10,7))

# Size and explsion
size_out = 3
size_in = 1
explode_out = (0.2,0.2)
explode_in = (0.3,0.3,0.3,0.3,0.3,0.3)

cmap = plt.get_cmap('tab20c')

outer_colors = cmap(np.array([8,0]))
inner_colors = cmap(np.array([11,10,9,3,2,1]))

patches1, texts1, autotexts1 = ax.pie(df_all.groupby(['Sex']).count().Name, radius=3, colors=outer_colors,
       labels=df_all.groupby(['Sex']).count().Name.index,autopct='%1.1f%%',pctdistance=0.85,
       wedgeprops=dict(width=size_out, edgecolor='black'),
       explode = explode_out)

patches2, texts2, autotexts2 = ax.pie(df_all.groupby(['Sex','Pclass']).count().Name, radius=2, colors=inner_colors,
       labels=[1,2,3,1,2,3],autopct='%1.1f%%', labeldistance=0.88,pctdistance=0.55,
       wedgeprops=dict(width=size_in, edgecolor='black'),
      explode = explode_in)

# Centre Cirle
centre_circle = plt.Circle((0,0),1.5,color='black', fc='white',linewidth=0)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

#plt.rcParams['font.size'] = 10.0
#plt.rc_context


# Define the labels on the outer plot
for t in texts1:
    t.set_size('large')
for t in autotexts1:
    t.set_size('large')
#autotexts1[0].set_color('y')


# Define the labels on the inner plot
for t in texts2:
    t.set_size('large')
for t in autotexts2:
    t.set_size('large')
#autotexts2[0].set_color('d')


# Setting legend
ax.legend(loc='lower right', bbox_to_anchor=(0.7, 0., 0.5, 0.5), shadow=1,title='Legend',
          handletextpad=1, labelspacing=0.5 , fontsize='12', labels=['female','male','1. class','2. class', '3. class','1. class','2. class', '3. class'])


ax.set(aspect="equal", title='Gender Distribution')
plt.axis('equal')
plt.show()

Here you can see the diffrent gender distributions to the ticket classes 1 to 3.

## Age Distribution by Ticket Class
Now the age distribution per ticket class will be visualized to get a better overview about the ages in each ticket class.

In [ ]:
# Sclicing the three classes
df_firstclass_ages = df_all[df_all.loc[:,'Pclass'] == 1]['Age'].copy()
df_secondclass_ages = df_all[df_all.loc[:,'Pclass'] == 2]['Age'].copy()
df_thirdclass_ages = df_all[df_all.loc[:,'Pclass'] == 3]['Age'].copy()

# Combining all classes in an array
df_all_class_ages =[df_firstclass_ages.values,
                    df_secondclass_ages.values,
                    df_thirdclass_ages.values]

# Font dictionary
font = {'color':  'black',
        'weight': 'normal',
        'size': 18,
}

# Building the figure and the axes for the plot
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 6) )

# plot violin plot
parts = axes.violinplot(df_all_class_ages
                   ,showmeans=False,
                    showmedians=True)
axes.set_title('Age Distribution per Class', fontdict=font, fontsize=25)

# Styling every violin in the graph
for pc in parts['bodies']:
    pc.set_facecolor('#FF8C00')
    pc.set_edgecolor('#000000')
    pc.set_linewidth(2)
    pc.set_alpha(0.7)


# adding horizontal grid lines
axes.yaxis.grid(True)
axes.set_xticks([y + 1 for y in range(len(df_all_class_ages))])
axes.set_xlabel('Class',fontdict=font, labelpad=20, size=20)
axes.set_ylabel('Age', fontdict=font,labelpad=20, size=20)


axes.vlines(1, df_firstclass_ages.describe()['25%'], df_firstclass_ages.describe()['75%'], color=['#000000'], linestyle='-', lw=5)
axes.vlines(2, df_secondclass_ages.describe()['25%'], df_secondclass_ages.describe()['75%'], color=['#000000'], linestyle='-', lw=5)
axes.vlines(3, df_thirdclass_ages.describe()['25%'], df_thirdclass_ages.describe()['75%'], color=['#000000'], linestyle='-', lw=5)
#axes.vlines(2, whiskersMin, whiskersMax, color='k', linestyle='-', lw=1)

# add x-tick labels
plt.setp(axes, xticks=[y + 1 for y in range(len(df_all_class_ages))],
         xticklabels=['First', 'Second','Third'])


plt.show()

The first class has a wide range of passengers with a mean age of nearly 40. This looks different in the second class. The passengers have at least the same range of ages but the mean age here is more in the area of 30 years. The same is with third class but here is the age range not so high as with the other classes. Its even much smaller outside the mean age of 25. The most passengers in the third class are around 25 years old. 

## Salutation Distribution


In [ ]:
df_all['Salutation']

# Survivor Distribution
In this chapter we will focus on the survivor distribution. In comparision to the **Feature Distribution** we will only focus on the training dataset because we only  have falid survival information in this dataset. The training dataset only includes 891 and not 1309 as in the **Feature Distribution**, therefore the distribution will look here slightly different.


## Male and Female Survivor
According to the distribution of 64.5% man and 35.6% women, I will now show the distribution of survivors broken down by gender. This distribution only shows the survival data for 891 passengers (from training data) and not for all ship passengers.  


In [ ]:
# The dataset with survived information
df_survivalinfo = pd.concat([df_all.loc[train_index,:], Survived], axis=1)

# Survival distribution per Sex
gp_survived_gender = df_survivalinfo.groupby(['Survived','Sex'])['Name'].count()[1]

# Gender Survival
gp_gender_survived = df_survivalinfo.groupby(['Sex','Survived']).count()['Name']

# Survival distribution Y/N 
gp_survived_yn = df_survivalinfo.groupby(['Survived']).agg('count')['Name']

# Survival total female / male 
gp_survival_total = df_survivalinfo.groupby(['Sex','Survived']).count().xs('Name', axis=1)


# Labels and size based on survival group by (df_survivalinfo)
#labels_suvinf = ['not suvived','survived']
sizes_suvinf = [gp_survived_yn[y] for y in range(len(gp_survived_yn))]
explode_suvinf = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

# Labels and size based on gender group by (gp_sex_survived)
labels_sexinf = [gp_survived_gender.index[y] for y in range(len(gp_survived_gender.index))]
sizes_sexinf = [gp_survived_gender[y] for y in range(len(gp_survived_gender))]
explode_sexinf = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

# Labels and size MALE Survivor Distribution
labels_maleinf = [gp_gender_survived['male'].index[y] for y in range(len(gp_gender_survived['male'].index))]
sizes_maleinf = [gp_gender_survived['male'][y] for y in range(len(gp_gender_survived['male']))]
explode_maleinf = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

# Labels and size FEMALE Survivor Distribution
labels_femaleinf = [gp_gender_survived['female'].index[y] for y in range(len(gp_gender_survived['female'].index))]
sizes_femaleinf = [gp_gender_survived['female'][y] for y in range(len(gp_gender_survived['female']))]
explode_femaleinf = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

# Labels and size total survival data
labels_totalsuvinf = [gp_survival_total.index[y] for y in range(len(gp_gender_survived.index))]
sizes_totalsuvinf = [gp_survival_total[y] for y in range(len(gp_survival_total))]
explode_totalsuvinf = (0.3, 0.0, 0.1, 0.0)  # not survived (fm), survived(fm), not survived(m), survived(m)


# Font dictionary
font = {'color':  'black',
        'weight': 'normal',
        #'size': 15,
        'fontsize':15
}

# Color maps for the pies
cmap = plt.get_cmap('tab20c')
survivedcolor = cmap(np.array([5,1]))
survivorallcolor = cmap(np.array([9,10,0,1]))

# Figure and axes of the plot / 4 * 2 plots 
gridsize = (4,2)
fig1 = plt.figure(figsize=(14,10))
ax1 = plt.subplot2grid(gridsize, (0,0))
ax2 = plt.subplot2grid(gridsize, (0,1))
ax3 = plt.subplot2grid(gridsize, (1,0))
ax4 = plt.subplot2grid(gridsize, (1,1))
ax5 = plt.subplot2grid(gridsize, (2,0), colspan= 2, rowspan= 2)

## fig1 configs
fig1.suptitle('Distributions of 891 Passengers (Trainingset)', fontsize=25)

## ax1 
# Define first pie for survival true falls
ax1.pie(sizes_suvinf, 
        explode=explode_suvinf,
        #labels=labels_suvinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=survivedcolor,
        labeldistance=1.15,
        pctdistance=0.55
       )
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
#ax1.fontdict=font
ax1.legend(loc='upper left',fontsize='12',labels=('not survived', 'survived'))
ax1.set_title('Survivor Distribution', fontdict=font, fontsize=20)

## ax2 
# Define second pie for sex to survival
ax2.pie(sizes_sexinf, 
        explode=explode_sexinf,
        #labels=labels_sexinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=outer_colors,
        labeldistance=1.15,
        pctdistance=0.55)
ax2.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
#ax2.fontdict=font
ax2.legend(loc='upper right', fontsize='12', labels=labels_sexinf )
ax2.set_title('Gender Survivors', fontdict=font, fontsize=20)

ax3.pie(sizes_maleinf, 
        explode=explode_maleinf,
        #labels=labels_sexinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=survivedcolor,
        labeldistance=1.15,
        pctdistance=0.55)
#ax3.fontdict=font
ax3.legend(loc='lower left', fontsize='12', labels=['not survived','survived'] )
ax3.set_title('Male Survivors', fontdict=font, fontsize=20)
ax3.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.


ax4.pie(sizes_femaleinf, 
        explode=explode_femaleinf,
        #labels=labels_sexinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=survivedcolor,
        labeldistance=1.15,
        pctdistance=0.55)
#ax4.fontdict=font
ax4.legend(loc='lower right', fontsize='12', labels=['not survived','survived'] )
ax4.set_title('Female Survivors', fontdict=font, fontsize=20)
ax4.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.


ax5.pie(sizes_totalsuvinf, 
        explode=explode_totalsuvinf,
        #labels=labels_totalsuvinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=survivorallcolor,
        labeldistance=1.15,
        pctdistance=0.55)
ax5.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
#ax4.fontdict=font
ax5.legend(loc='lower right', fontsize='12', labels=['female not survived','female survived','male not survived','male survived'] )
ax5.set_title('Overall Survivors', fontdict=font, fontsize=20)
ax5.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In total of the 891 passenger informations are 62% that did not made it out of the tragedy and died in the open water.  These survivors are divided into 68.1% women and 31.9% man. It is clearly visible that a lot of female passengers have been rescued. If we look closer into the male passengers data (third pie chart: "Male Survivors), 81% of the male passengers did not survive. In conctract, 74% female passengers survived. That's according to the gentleman's behavior of that time. This situation is best described by a quote of sir ....


68% man and 32% women. So at least a lot of male passengers on board. 



_______________________
https://seaborn.pydata.org/examples/many_facets.html
_______________________


## Distribution of  Ticket Class to Survival


In [ ]:
# Grouped by Survived and Pclass
gp_survpclass = df_survivalinfo.groupby(['Survived','Pclass'])['Name'].count()

gridsize = (1,2)
fig1 = plt.figure(figsize=(12,8))
ax1 = plt.subplot2grid(gridsize, (0,0), colspan=2, rowspan=1)

# Bar chart design
bar_width = 0.35
cmap = plt.get_cmap('tab20b')

survbarcol = cmap(np.array([7]))
nsurvbarcol = cmap(np.array([12]))

barindex = gp_survpclass[0].index   # Group by index of bar plot data
xtickslables = [barindex[y] for y in range(len(gp_survpclass[0].index))]



ax1.bar(barindex
        ,gp_survpclass[0].values
        ,bar_width
        ,color=nsurvbarcol
        )

ax1.bar(barindex + bar_width
        ,gp_survpclass[1].values
        ,bar_width
        ,color=survbarcol)

ax1.set_xlabel('Ticket Class')
ax1.set_ylabel('Passenger Count')
ax1.set_title('Suvivors per Ticket Class')
ax1.set_xticks(barindex + bar_width / 2)
ax1.set_xticklabels(xtickslables)
ax1.legend(labels=('not survived','survived'))

fig.tight_layout()
plt.show()

In [ ]:
ptbl = pd.DataFrame.pivot_table(df_survivalinfo, values=['Fare', 'Survived'], index=['Pclass'],
                     aggfunc={'Survived': ['sum'], 'Fare': [min,max,np.mean]})

ptbl


## Survivors by Salutation

In [ ]:
#df_all.loc[train_index,:].groupby('Salutation').count()


df_survival_sal = df_survivalinfo.groupby(['Salutation','Survived'])['Name'].count()
df_survival_sal
#df_all.loc[train_index,:].groupby('Salutation').count()['Name']

#.index

#pd.pivot_table(df_survivalinfo, values='Survived', index=['Salutation'],columns=['Survived'], aggfunc=np.sum)

# values='D',
#['Name']
#df_survivalinfo.pivot(index='Salutation', columns='Name', values='Survived')

In [ ]:
df_all[df_all['Salutation']=='Capt.']

In [ ]:
sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(6, 15))

# Load the example car crash dataset
#crashes = sns.load_dataset("car_crashes").sort_values("total", ascending=False)
salutations = df_survivalinfo.groupby(['Salutation','Survived']).count()

# Plot the total crashes
sns.set_color_codes("pastel")
sns.barplot(x="Name", y=, data=salutations,
            label="Name", color="b")

# Plot the crashes where alcohol was involved
sns.set_color_codes("muted")
sns.barplot(x="alcohol", y="abbrev", data=salutations,
            label="Alcohol-involved", color="b")

# Add a legend and informative axis label
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0, 24), ylabel="",
       xlabel="Automobile collisions per billion miles")
sns.despine(left=True, bottom=True)

## Gaussian distrbution for age and fare price zu survived = 1

https://matplotlib.org/gallery/statistics/histogram_multihist.html#sphx-glr-gallery-statistics-histogram-multihist-py

Am Ende müssen sich alle Eingerenzungen durch die Visualisierungen in den Prediction Model wiederfinden.
(z.B.: min age, max age, mean age, in der Klasse xyz sorgt dafpr, dass du eher überlebts ...)

In [ ]:
fare_survived = df_survivalinfo[df_survivalinfo['Survived'] == 1]['Fare']
fare_survived = df_survivalinfo['Fare']

fare_survived.describe()['mean']
fare_survived.describe()['std']

fare_survived.describe()
fare_survived.values

age_survived = df_survivalinfo['Age']

fare_survived

In [ ]:
inputfeature = fare_survived

mu = inputfeature.describe()['mean']  # mean of distribution
sigma = inputfeature.describe()['std']  # standard deviation of distribution
x = mu + sigma * inputfeature.values

num_bins = 50

fig, ax = plt.subplots(figsize=(12,7))

# the histogram of the data
n, bins, patches = ax.hist(inputfeature,num_bins, density=1)

# add a 'best fit' line
y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
     np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
ax.plot(bins, y, '--')
ax.set_xlabel('Fare Price')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of IQ: $\mu=100$, $\sigma=15$')

# Tweak spacing to prevent clipping of ylabel
#fig.tight_layout()
plt.show()

In [ ]:
df_survivalinfo[df_survivalinfo['Survived'] == 1].describe()

## Survivor by Fare Price 


In [ ]:
# Pivot for age and fare
ptbl_survived = pd.DataFrame.pivot_table(df_survivalinfo, values=['Fare', 'Age', 'Survived'], index=['Sex', 'Pclass'],
                     aggfunc={'Fare': np.mean,'Age': [min, max, np.mean], 'Survived': ['sum']})
ptbl_survived

### Ticketprice to Survival

### Scatter plot mit zwei features und dem dritten als Farbe der dots und größe der dots.


## Overall Bivariate Relation

In [ ]:
sns.pairplot(df_survivalinfo, hue='Survived')

An overview about the 

# Model

## Man müsste einen K-Nearest Neighbor Ansatz auf Basis der Ticketklasse, dem Geschlecht und vlt noch dem Alter aufbauen, da Ticketklasse und Geschlecht erheblichen Einfluss auf das Überleben hatten. Siehe oben die Skizzen


In [ ]:

df_all.loc[train_index,:]

In [ ]:
# One hot encoding for the categorical data
df_train_hot  = df_all.loc[train_index,:].copy()
df_train_hot = df_train_hot.drop(['Name','Ticket','Cabin'], axis=1)
df_train_hot = pd.get_dummies(df_train_hot, columns=['Sex','Embarked'])

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train_hot, Survived, test_size = 0.18, random_state = 25)  ## 50  25 ## 0.25 25

In [ ]:
# K-Nearest Neighbours
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


X_train = X_train
y_train = y_train

KNNC = KNeighborsClassifier(n_neighbors=16)
KNNC.fit(X_train, y_train)

y_pred = KNNC.predict(X_test)

# Summary of the predictions made by the classifier
print(classification_report(y_test, y_pred, target_names=['0','1']))


# Accuracy score

#print(y_test, y_pred)
#print(y_test, y_pred)

#print('accuracy is',accuracy_score(y_pred,y_test))

print ("Models accuracy score: ", accuracy_score(y_test, y_pred))


In [ ]:
from yellowbrick.classifier import ClassificationReport

classes = ["will not suvive", "will survive"]

##KNNCA = KNeighborsClassifier()

# Instantiate the classification model and visualizer
visualizer = ClassificationReport(KNNC, classes=classes, support=True)

visualizer.fit(X_train, y_train)  # Fit the visualizer and the model
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
g = visualizer.poof()             # Draw/show/poof the data

In [ ]:
# confusion matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score

# creating odd list of K for KNN
myList = list(range(1,50))

# subsetting just the odd ones
#neighbors = list(filter(lambda x: x % 2 != 0, myList))
neighbors = list(myList)


# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

In [ ]:
# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print ("The optimal number of neighbors is %d" % optimal_k)

# plot misclassification error vs k
plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
neighbors
optimal_k
MSE

In [ ]:
#MSE.index(min(MSE))

#print(neighbors)

#list(neighbors)
#print(less_than_zero)
print(lambda x: x % 2 != 0)

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)


#N = 50
y = df_survivalinfo.loc[train_index]['Fare'].values
x = df_survivalinfo.loc[train_index]['Age'].values
z = df_survivalinfo.loc[train_index]['Survived'].values
#colors = np.random.rand(N)
#area = (30 * np.random.rand(N))**2  # 0 to 15 point radii


###for surv in z:
###    if surv == 1:
###        plt.plot(x, y, 'o', markerfacecolor='r',
###            markeredgecolor='k', markersize=3)

   
plt.scatter(x, y) 
            #s=area, c=colors, alpha=0.5)
plt.show()

In [ ]:

#N = 50
x = df_all.loc[train_index]['Pclass'].values
y = df_all.loc[train_index]['Fare'].values
#colors = np.random.rand(N)
#area = (30 * np.random.rand(N))**2  # 0 to 15 point radii

plt.scatter(x, y) 
            #s=area, c=colors, alpha=0.5)
plt.show()

In [ ]:
df_survivalinfo[['Pclass','Age']]

# Only training data
df_all.loc[train_index][['Pclass','Age']]

df_all.loc[train_index][['Fare','Age']].values
df_all.loc[train_index]



In [ ]:
### K - Means

print(__doc__)

# Author: Phil Roth <mr.phil.roth@gmail.com>
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

plt.figure(figsize=(12, 12))

n_samples = 800
random_state = 170
#X, y = make_blobs(n_samples=n_samples, random_state=random_state)

X = df_all.loc[train_index][['Age','Fare']].values
y = Survived

# Incorrect number of clusters
y_pred = KMeans(n_clusters=4, random_state=random_state).fit_predict(X)

plt.subplot(221)
plt.scatter(X[:, 0], X[:, 1], c=y_pred)
plt.title("Incorrect Number of Blobs")

# Anisotropicly distributed data
transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]
X_aniso = np.dot(X, transformation)
y_pred = KMeans(n_clusters=5, random_state=random_state).fit_predict(X_aniso)

plt.subplot(222)
plt.scatter(X_aniso[:, 0], X_aniso[:, 1], c=y_pred)
plt.title("Anisotropicly Distributed Blobs")

# Different variance
X_varied, y_varied = make_blobs(n_samples=n_samples,
                                cluster_std=[1.0, 2.5, 0.5],
                                random_state=random_state)
y_pred = KMeans(n_clusters=3, random_state=random_state).fit_predict(X_varied)

plt.subplot(223)
plt.scatter(X_varied[:, 0], X_varied[:, 1], c=y_pred)
plt.title("Unequal Variance")

# Unevenly sized blobs
X_filtered = np.vstack((X[y == 0][:500], X[y == 1][:100], X[y == 2][:10]))
y_pred = KMeans(n_clusters=3,
                random_state=random_state).fit_predict(X_filtered)

plt.subplot(224)
plt.scatter(X_filtered[:, 0], X_filtered[:, 1], c=y_pred)
plt.title("Unevenly Sized Blobs")

plt.show()

In [ ]:
### K nearest Neighbour

print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

n_neighbors = 10

# import some data to play with
iris = datasets.load_iris()

# we only take the first two features. We could avoid this ugly
# slicing by using a two-dim dataset
X = df_all.loc[train_index][['Pclass','Age']].values
y = Survived
##X = iris.data[:, :2]
##y = iris.target

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    #clf = neighbors.RadiusNeighborsClassifier(n_neighbors, weights=weights)
    
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("2-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, weights))

plt.show()

In [ ]:
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

X = StandardScaler().fit_transform(X)
X

In [ ]:
print(__doc__)

import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler


# #############################################################################
# Generate sample data
#centers = [[1, 1], [-1, -1], [1, -1]]
#X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
#                            random_state=0)

#X = StandardScaler().fit_transform(X)

X = df_all.loc[train_index][['Fare','Age']].values


# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

##print('Estimated number of clusters: %d' % n_clusters_)
##print('Estimated number of noise points: %d' % n_noise_)
##print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
##print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
##print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
##print("Adjusted Rand Index: %0.3f"
##      % metrics.adjusted_rand_score(labels_true, labels))
##print("Adjusted Mutual Information: %0.3f"
##      % metrics.adjusted_mutual_info_score(labels_true, labels))
##print("Silhouette Coefficient: %0.3f"
##      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

## REAL DBSCAN Part

In [ ]:
## Data preparation part ###
df_survivalinfo_onehot  = df_survivalinfo.copy()
df_survivalinfo_onehot = pd.get_dummies(df_survivalinfo_onehot, columns=['Sex'], prefix = ['Sex'])


### DBSCAN part ###
print(__doc__)

import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler


# #############################################################################
# Generate sample data
#centers = [[1, 1], [-1, -1], [1, -1]]
#X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
#                            random_state=0)

#X = StandardScaler().fit_transform(X)

#X = df_all.loc[train_index][['Fare','Age','Pclass', 'Family','SibSp','Parch']].values
#X = df_all[['Fare','Age','Pclass', 'Family',]].values
#X = df_survivalinfo[['Fare','Age','Pclass', 'Family','Survived']].values
#X = df_survivalinfo[['Age','Survived','Fare','Pclass', 'Family']].values
#####X = df_survivalinfo_onehot[['Fare','Age','Pclass', 'Family','Survived','Sex_female','Sex_male']].values
#X = df_survivalinfo_onehot[['Pclass','Age','Fare','Sex_female','Sex_male','Family','Survived']].values
#,'Sex_female','Sex_male','Family'

X = df_survivalinfo_onehot[['Fare','Age']].values

# #############################################################################
# Compute DBSCAN  ### 10 25 ist gut
db = DBSCAN(eps=10, min_samples=5, algorithm='auto').fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
##print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
##print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
##print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
##print("Adjusted Rand Index: %0.3f"
##      % metrics.adjusted_rand_score(labels_true, labels))
##print("Adjusted Mutual Information: %0.3f"
##      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Figure size set
plt.figure(figsize=(12,7))

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)

plt.show()

In [ ]:
#pd.concat(,db.labels_)
    
df_survivalinfo


In [ ]:
sns.pairplot(df_survivalinfo, hue='Survived')

In [ ]:
# seaborn's kdeplot, plots univariate or bivariate density estimates.
#Size can be changed by tweeking the value used
sns.FacetGrid(df_survivalinfo, hue="Survived", height=5).map(sns.kdeplot, "Pclass").add_legend()
plt.show()

# Schrottsammler



In [ ]:
## CABIN TRY #####




# Creating a data frame for the Cabin values to split multiple values into seperate columns
df_cabin_expand = df_all.loc[:,'Cabin'].str.split(' ', expand=True)

# While I transfered the cabin information to a string value to afterwards split the single list values.
# Unfortunately, all nan fields were also converted. Therefore I cannot use the notna or notnull function to determin the ones with
# filled information.
# To fix that problem I will replace all NaN strings with a NaN object.

df_cabin_expand = df_cabin_expand.replace(['NaN','None'],np.nan)


df_all.loc[:,'Cabin'].str.split(' ', expand=True)

### Violin Plotts with Seaborn

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 8) )

# Font dictionary
font = {'color':  'black',
        'weight': 'normal',
        'size': 18,
}

sns.set(style="darkgrid", font_scale=2)

ax = sns.violinplot(x='Pclass', y='Age', data=df_all )

ax.set_title('Age Distribution per Class', fontsize=25)
ax.set_xlabel('Class',fontdict=font, labelpad=20, fontsize=20)
ax.set_ylabel('Age', fontdict=font,labelpad=20, fontsize=20)


### Age to Survived

https://matplotlib.org/gallery/statistics/boxplot_vs_violin.html#sphx-glr-gallery-statistics-boxplot-vs-violin-py

min age <br>
max age  <br>
mean age <br>

per class

boxplot je Pclass mit min age, max age, mean age

In [ ]:
df_two_features = [df_all['Age'].values,df_all['Fare'].values ]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))

# plot violin plot
axes.violinplot(df_two_features
                   ,showmeans=False,
                    showmedians=True)
axes.set_title('Age and Fare Distribution')

# adding horizontal grid lines
axes.yaxis.grid(True)
axes.set_xticks([y + 1 for y in range(len(df_two_features))])
axes.set_xlabel('Two separate features')
axes.set_ylabel('Observed values Age / Fare')

# add x-tick labels
plt.setp(axes, xticks=[y + 1 for y in range(len(df_two_features))],
         xticklabels=['Age', 'Fare'], )

plt.ylim(top=150)  # adjust the top leaving bottom unchanged
plt.ylim(bottom=-20)  # adjust the bottom leaving top unchanged

plt.show()

### 4 Pie charts

In [ ]:
# The dataset with survived information
df_survivalinfo = pd.concat([df_all, Survived], axis=1)

# Survival distribution per Sex
gp_survived_gender = df_survivalinfo.groupby(['Survived','Sex'])['Name'].count()[1]

# Gender Survival
gp_gender_survived = df_survivalinfo.groupby(['Sex','Survived']).count()['Name']

# Survival distribution Y/N 
gp_survived_yn = df_survivalinfo.groupby(['Survived']).agg('count')['Name']



# Labels and size based on survival group by (df_survivalinfo)
#labels_suvinf = ['not suvived','survived']
sizes_suvinf = [gp_survived_yn[y] for y in range(len(gp_survived_yn))]
explode_suvinf = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

# Labels and size based on gender group by (gp_sex_survived)
labels_sexinf = [gp_survived_gender.index[y] for y in range(len(gp_survived_gender.index))]
sizes_sexinf = [gp_survived_gender[y] for y in range(len(gp_survived_gender))]
explode_sexinf = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

# Labels and size MALE Survivor Distribution
labels_maleinf = [gp_gender_survived['male'].index[y] for y in range(len(gp_gender_survived['male'].index))]
sizes_maleinf = [gp_gender_survived['male'][y] for y in range(len(gp_gender_survived['male']))]
explode_maleinf = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

# Labels and size FEMALE Survivor Distribution
labels_femaleinf = [gp_gender_survived['female'].index[y] for y in range(len(gp_gender_survived['female'].index))]
sizes_femaleinf = [gp_gender_survived['female'][y] for y in range(len(gp_gender_survived['female']))]
explode_femaleinf = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')


# Font dictionary
font = {'color':  'black',
        'weight': 'normal',
        #'size': 15,
        'fontsize':15
}

cmap = plt.get_cmap('tab20c')
survivedcolor = cmap(np.array([5,1]))

# Figure and axes1 and axes2 of the plot
gridsize = (4,2)
fig1 = plt.figure(figsize=(18,14))
ax1 = plt.subplot2grid(gridsize, (0,0))
ax2 = plt.subplot2grid(gridsize, (0,1))
ax3 = plt.subplot2grid(gridsize, (1,0))
ax4 = plt.subplot2grid(gridsize, (1,1))


## ax1 
# Define first pie for survival true falls
ax1.pie(sizes_suvinf, 
        explode=explode_suvinf,
        #labels=labels_suvinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=survivedcolor,
        labeldistance=1.15,
        pctdistance=0.55
       )
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax1.fontdict=font
ax1.legend(loc='upper left',fontsize='small',labels=('not survived', 'survived'))
ax1.set_title('Survivor Distribution', fontdict=font, fontsize=20)

## ax2 
# Define second pie for sex to survival
ax2.pie(sizes_sexinf, 
        explode=explode_sexinf,
        #labels=labels_sexinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=outer_colors,
        labeldistance=1.15,
        pctdistance=0.55)
ax2.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax2.fontdict=font
ax2.legend(loc='upper right', fontsize='small', labels=labels_sexinf )
ax2.set_title('Gender Survivor Distribution', fontdict=font, fontsize=20)

ax3.pie(sizes_maleinf, 
        explode=explode_maleinf,
        #labels=labels_sexinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=survivedcolor,
        labeldistance=1.15,
        pctdistance=0.55)
ax3.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax3.fontdict=font
ax3.legend(loc='lower left', fontsize='small', labels=['not survived','survived'] )
ax3.set_title('Male Survivor Distribution', fontdict=font, fontsize=20)
ax3.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.


ax4.pie(sizes_femaleinf, 
        explode=explode_femaleinf,
        #labels=labels_sexinf,
        autopct='%1.1f%%',
        shadow=True, startangle=90,
        colors=survivedcolor,
        labeldistance=1.15,
        pctdistance=0.55)
ax4.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax4.fontdict=font
ax4.legend(loc='lower right', fontsize='small', labels=['not survived','survived'] )
ax4.set_title('Female Survivor Distribution', fontdict=font, fontsize=20)
ax4.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

#plt.rcParams['font.size'] = 13.0
#plt.rc_context


plt.show()

## Ein weiterer Violin Plot

In [ ]:
df_two_features = [df_all['Age'].values,df_all['Fare'].values ]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))

# plot violin plot
axes.violinplot(df_two_features
                   ,showmeans=False,
                    showmedians=True)
axes.set_title('Age and Fare Distribution')

# adding horizontal grid lines
axes.yaxis.grid(True)
axes.set_xticks([y + 1 for y in range(len(df_two_features))])
axes.set_xlabel('Two separate features')
axes.set_ylabel('Observed values Age / Fare')

# add x-tick labels
plt.setp(axes, xticks=[y + 1 for y in range(len(df_two_features))],
         xticklabels=['Age', 'Fare'], )

plt.ylim(top=150)  # adjust the top leaving bottom unchanged
plt.ylim(bottom=-20)  # adjust the bottom leaving top unchanged

plt.show()

In [ ]:
#df_fristclass_ages
#test= pd.DataFrame(df_all[df_all.loc[:,'Pclass'] == 1]['Age'].copy())
#test

firstclass = pd.DataFrame(df_all[df_all.loc[:,'Pclass'] == 1]['Age'].copy())
secondclass = pd.DataFrame(df_all[df_all.loc[:,'Pclass'] == 2]['Age'].copy())
thirdclass = pd.DataFrame(df_all[df_all.loc[:,'Pclass'] == 3]['Age'].copy())

df_firstclass_ages.rename

# violin plot
frame = [df_firstclass_ages
    ,df_secondclass_ages
    ,df_thirdclass_ages]

results = pd.concat(frame,axis =1)

results.columns = ['Firstclass_Age','Secondclass_Age','Thirdclass_Age']

In [ ]:
# violin plot
frame = [df_firstclass_ages
    ,df_secondclass_ages
    ,df_thirdclass_ages]

results = pd.concat(frame,axis =1)
results.rename(index=str, columns={"Age": "Age Firstclass"})

In [ ]:
# Concat Survived and training set 
fig = plt.figure(figsize=(12,10))
#gs = fig.add_gridspec(2, 2)
#ax1 = fig.add_subplot(gs[0, 0])
ax = fig.add_subplot()



df_all_survived = pd.concat([df_all.loc[train_index,:],Survived], axis=1)
df_all_survived 


In [ ]:
n_groups = 10

# Mans value count per type
means_men = df_all.groupby(['Sex']).count().loc['male',:].values
#std_men = (2, 3, 4, 1, 2)

# Womens value count per type
means_women = df_all.groupby(['Sex']).count().loc['female',:].values
#std_women = (3, 5, 2, 3, 3)



fig, ax = plt.subplots(figsize=(10,6))


index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.4
error_config = {'ecolor': '0.3'}

rects1 = ax.bar(index, means_men, bar_width,
                alpha=opacity, color='b',
                error_kw=error_config,
                label='Men')

rects2 = ax.bar(index + bar_width, means_women, bar_width,
                alpha=opacity, color='g',
                error_kw=error_config,
                label='Women')

ax.set_xlabel('Group')
ax.set_ylabel('Count')
ax.set_title('Scores by group and gender')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(('Pclass','Name','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked','Family'))
ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
df_all.groupby(['Sex','Pclass']).count().index
df_all.groupby(['Sex','Pclass']).count()['female']
#index.levels[1]
#df_all.groupby(['Sex','Pclass']).count().index.levels[0]

In [ ]:
#df_all.groupby(['Sex']).count().Name
#df_all.groupby(['Sex','Pclass']).count().Name

df_all.groupby(['Sex','Pclass']).count().loc['female']

#df_all.groupby(['Sex','Pclass']).count().loc

#.loc['male']



In [ ]:
#df_all.pivot(columns='Sex', values=['Fare','Pclass'])
df_all.pivot(columns='Pclass', values=['Fare']) # very good one


In [ ]:
#df_all['Sex'].values
#Survived.values
df_all.groupby(['Sex']).count().Name.male

In [ ]:

#fig = plt.figure() # figure
#fig.suptitle('Gender Seperation on 'Titanic')

fig, ax = plt.subplots(1,1)
#my_plotter(ax,df_train['Sex'].values, Survived.values, {: 'x'})
my_bar(ax, df_train['Sex'].values, Survived.values, {'facecolor':'r'})


In [ ]:
#df_all[['Sex','Age','SibSp','Parch','Embarked','Family']].hist(Survived)
pd.concat([df_train, Survived], axis=1).hist()
#.hist()


In [ ]:
df_all.groupby(['Sex']).count().loc['female',:].values

In [ ]:
df_all.pivot("Pclass", "Age", "Sex")

#passengers = sns.load_dataset("flights")
#flights = df_all.pivot("Pclass", "Age", "Sex")
#ax = sns.heatmap(flights)

In [ ]:
df_all[(df_all['Ticket'].str.contains('PC', regex=False) == True)]
